# Ceneo Scraper

## Załadowanie bibliotek

In [ ]:
import requests
from bs4 import BeautifulSoup

## Wysłanie zapytania do serwera

In [ ]:
product_id = "138331381"
url = f"https://www.ceneo.pl/{product_id}#tab=reviews_scroll"
response = requests.get(url)
response.status_code

# Pobranie opinii z kodu HTML strony

In [ ]:
page_dom = BeautifulSoup(response.text, "html.parser")
opinions = page_dom.select("div.js_product-review")
print(type(opinions))
opinion = opinions.pop(0)
print(type(opinion))

In [ ]:
print(opinion)

# Ekstrakcja składowych pojedynczej opinii

|Składowa|Selektor|Zmienna|
|--------|--------|-------|
|id opinii|[data-entry-id]|opinion_id|
|autor|span.user-post__author-name|author|
|rekomendacja|span.user-post__author-recomendation|recommendation|
|gwiazdki|span.user-post__score-count|stars|
|treść|div.user-post__text|content|
|lista zalet|.review-feature__title ~ .review-feature__title--positives|pros|
|lista wad|  .review-feature__title ~ .review-feature__title--negatives |cons|
|dla ilu przydatna|button.vote-yes > span|helpful|
|dla ilu nieprzydatna| button.vote-no >span  |unhelpful|
|data wystawenia|span.user-post__published > time:nth-child(1)[datetime]|publish_date|
|data zakupu|span.user-post__published > time:nth-child(2)[datetime]|purchase_date|

In [ ]:
opinion_id = data-entry-id